# GET SIFT

In [28]:
import numpy as np
import pandas as pd
import cv2
import matplotlib.pyplot as plt
import os
import glob
from sklearn.decomposition import PCA
import ast

from sklearn.cluster import MiniBatchKMeans
import time
from collections import Counter
import pickle
from tempfile import TemporaryFile
from sklearn.cluster import KMeans
%matplotlib inline
import natsort
from sklearn import preprocessing
import copy

In [10]:
def SIFT_dataset(path,dataset):
    descriptor_list=[]
    for video_id in dataset['Index']:
        path_train=os.path.join(path, str(video_id))
        images=[name for name in os.listdir(path_train) if os.path.isfile(os.path.join(path_train, name))]
        images=natsort.natsorted(images)
        num_images=len(images)
        if num_images>40:
            
            print(num_images)
       
        if num_images>25:
            images=images[5:25]
        if num_images<=25:
            images=images[0:20]
        for name in images:
            file=os.path.join(path_train, str(name))
            img_building = cv2.imread(file)
            
#             plt.figure(figsize=(312,128))
#             plt.imshow(img_building)
#             plt.show()

            img_building = cv2.cvtColor(img_building, cv2.COLOR_BGR2GRAY)   # Convert from cv's BRG default color order to HSV
            sift = cv2.xfeatures2d.SIFT_create()
            step_size = 15

            kp = [cv2.KeyPoint(x, y, step_size) for x in range(0, 312, step_size) for y in range(0, 128, step_size)]
            img2 = cv2.drawKeypoints(img_building, kp, None, color=(0,255,0), flags=0)
#             plt.figure(figsize=(312,128))
#             plt.imshow(img2), plt.show()
            
            descriptor = sift.compute(img_building, kp)[1]
            descriptor_list.append(descriptor)
            

    return descriptor_list

In [29]:
train=pd.read_csv(r'/storage/demo_dataset/train_set.csv')
test=pd.read_csv(r'/storage/demo_dataset/test_set.csv')
val=pd.read_csv(r'/storage/demo_dataset/val_set.csv')

path_train ='/storage/demo_dataset/train/'
path_test ='/storage/demo_dataset/test/'
path_val ='/storage/demo_dataset/val/'

In [12]:
#get train
descriptor_list_train=SIFT_dataset(path_train,train)
print("descriptor_list_train:",len(descriptor_list_train))
with open(r'/storage/demo_dataset/train_sift.pickle', 'wb') as f:
    pickle.dump(descriptor_list_train, f, protocol=5)

42
67
51
41


TypeError: object of type 'NoneType' has no len()

In [6]:
#get test
descriptor_list_test=SIFT_dataset(path_test,test)
print("descriptor_list_test:",len(descriptor_list_test))
with open(r'/storage/demo_dataset/test.pickle', 'wb') as f:
    pickle.dump(descriptor_list_test, f)

descriptor_list_test: 9840


In [7]:
#get validation
descriptor_list_val=SIFT_dataset(path_val,val)
print("descriptor_list_val:",len(descriptor_list_val))

with open(r'/storage/demo_dataset/val_sift.pickle', 'wb') as f:
    pickle.dump(descriptor_list_val, f)

descriptor_list_val: 9860


### load dataset 

In [30]:
with open(r'/storage/demo_dataset/train.pickle', 'rb') as f:
    descriptor_list_train = pickle.load(f)
print("descriptor_list_train:",len(descriptor_list_train))

with open(r'/storage/demo_dataset/test.pickle', 'rb') as f:
    descriptor_list_test = pickle.load(f)
print("descriptor_list_test:",len(descriptor_list_test))

with open(r'/storage/demo_dataset/val_sift.pickle', 'rb') as f:
    descriptor_list_val = pickle.load(f)    
print("descriptor_list_val:",len(descriptor_list_val))

descriptor_list_train: 79000
descriptor_list_test: 9840
descriptor_list_val: 9860


In [ ]:
def all_train_desc(descriptor_list):
    all_train_desc = []
    for i in range(len(descriptor_list)):
        for j in range(descriptor_list[i].shape[0]):
            all_train_desc.append(descriptor_list[i][j,:])

    all_train_desc = np.array(all_train_desc)
    return all_train_desc


# form training set histograms for each training image using BoW representation
def formTrainingSetHistogram(x_train, kmeans, k):
    train_hist = []
    for i in range(len(x_train)):
        data = copy.deepcopy(x_train[i])
        predict = kmeans.predict(data)
        train_hist.append(np.bincount(predict, minlength=k).reshape(1,-1).ravel())
        
    return np.array(train_hist)

def hist (descriptor_list, kmeans, k):
    hist = formTrainingSetHistogram(descriptor_list, kmeans, k)
    scaler = preprocessing.StandardScaler().fit(hist)
    hist = scaler.transform(hist)
    return hist

In [ ]:
all_train_desc = all_train_desc(descriptor_list_train)

In [ ]:
km = KMeans(n_clusters=30, random_state=456).fit(all_train_desc)  

In [ ]:
with open(r'/storage/demo_dataset/kmeans.pickle', 'wb') as f:
    pickle.dump(km, f)

In [ ]:
with open(r'/storage/demo_dataset/kmeans.pickle', 'rb') as f:
    kmeans = pickle.load(f)    

In [16]:
# form training set histograms for each training image using BoW representation
def formTrainingSetHistogram(x_train, kmeans, k):
    train_hist = []
    for i in range(len(x_train)):
        data = copy.deepcopy(x_train[i])
        predict = kmeans.predict(data)
        train_hist.append(np.bincount(predict, minlength=k).reshape(1,-1).ravel())
        
    return np.array(train_hist)


In [17]:
train_hist = hist(descriptor_list_train, kmeans, 30)
test_hist = hist(descriptor_list_test, kmeans, 30)
val_hist   = hist(descriptor_list_val, kmeans, 30)

In [18]:
print(train_hist.shape)
print(test_hist.shape)
print(val_hist.shape)

(79000, 30)
(9840, 30)
(9860, 30)


In [31]:
with open(r'/storage/demo_dataset/train_hist.pickle', 'rb') as f:
    train_hist = pickle.load(f)

with open(r'/storage/demo_dataset/test_hist.pickle', 'rb') as f:
    test_hist= pickle.load(f)

with open(r'/storage/demo_dataset/val_hist.pickle', 'rb') as f:
    val_hist = pickle.load(f)    

## Change into video-level 

In [32]:
def feature_video(hist):
    train=[]
    for index in range(0,len(hist),20):
        result =np.concatenate(hist[index:index+20], axis=0).reshape(20,-1)
        train.append(result)
    return train

In [33]:
train_video=feature_video(train_hist)
test_video=feature_video(test_hist)
val_video=feature_video(val_hist)

In [34]:
print('train_video:',len(train_video),'test_video:',len(test_video),'val_video:',len(val_video))

train_video: 3950 test_video: 492 val_video: 493


In [35]:
train_video[0].shape

(20, 30)

### Dataloader

In [36]:
import time
import numpy as np
from torchvision import datasets
from torchvision import transforms
from torch.utils.data import DataLoader
import torch.nn.functional as F
import torch
from torch.utils.data import Dataset
if torch.cuda.is_available():
    torch.backends.cudnn.deterministic = True
    
import os
import pandas as pd
import time
import random
from PIL import Image
from torch.utils.data import Dataset
import matplotlib.pyplot as plt

In [37]:
label_dict={'No gesture':0, 
            'Pulling Hand In':1, 
            'Pushing Two Fingers Away':2,
            'Rolling Hand Forward':3, 
            'Shaking Hand':4, 
            'Sliding Two Fingers Down':5,
            'Sliding Two Fingers Left':6, 
            'Sliding Two Fingers Right':7,
            'Thumb Up':8, 
            'Turning Hand Counterclockwise':9}
train_label=np.array(list((map(label_dict.get, train['Label']))))
test_label=np.array(list((map(label_dict.get,test['Label']))))
val_label=np.array(list((map(label_dict.get,val['Label']))))

In [38]:
class Feature_hist_dataset(Dataset):
    """Custom Dataset for loading FashionMNIST images"""

    def __init__(self, labels, hist,transform):
        
        self.hist=torch.from_numpy(hist).float()
        self.y =torch.from_numpy(labels).float()
        self.transform = transform

    def __getitem__(self, index):
    
        label = self.y[index]
        hist_per_video=self.hist[index]
        return hist_per_video, label

    def __len__(self):
        return self.y.shape[0]

In [39]:
train_dataset = Feature_hist_dataset(labels=train_label,hist=np.array(train_video),transform=None)
test_dataset = Feature_hist_dataset(labels=test_label,hist=np.array(test_video),transform=None)
val_dataset = Feature_hist_dataset(labels=val_label,hist=np.array(val_video),transform=None)

In [101]:
### SETTINGS
##########################

# Device
device = torch.device("cuda:0" if torch.cuda.is_available() else "cpu")

# Hyperparameters
random_seed = 1
learning_rate = 0.01
num_epochs = 12
batch_size = 32

# Architecture
num_features = 20*30
num_hidden_1 = 128
num_hidden_2 = 256
num_classes = 10

In [102]:
train_loader = DataLoader(dataset=train_dataset,
                          batch_size=batch_size, 
                          shuffle=True,
                         num_workers=8)

test_loader = DataLoader(dataset=test_dataset, 
                         batch_size=batch_size, 
                         shuffle=False,
                        num_workers=8)

val_loader=DataLoader(dataset=val_dataset, 
                         batch_size=batch_size, 
                         shuffle=False,
                        num_workers=8)


## MLP

In [103]:
##########################
### MODEL
##########################

class MultilayerPerceptron(torch.nn.Module):

    def __init__(self, num_features, num_classes):
        super(MultilayerPerceptron, self).__init__()
        
        ### 1st hidden layer
        self.linear_1 = torch.nn.Linear(num_features, num_hidden_1)
        self.linear_1_bn = torch.nn.BatchNorm1d(num_hidden_1)
        torch.nn.Dropout(0.2)
        
        ### 2nd hidden layer
        self.linear_2 = torch.nn.Linear(num_hidden_1, num_hidden_2)
        self.linear_2_bn = torch.nn.BatchNorm1d(num_hidden_2)
        
        ### Output layer
        self.linear_out = torch.nn.Linear(num_hidden_2, num_classes)
        self.linear_out.weight.detach().normal_(0.0, 0.1)
        self.linear_out.bias.detach().zero_()
        
    def forward(self, x):
        out = self.linear_1(x)
        # note that batchnorm is in the classic
        # sense placed before the activation
        out = self.linear_1_bn(out)
        out = F.relu(out)
        
        out = self.linear_2(out)
        out = self.linear_2_bn(out)
        out = F.relu(out)
        
        logits = self.linear_out(out)
        probas = F.softmax(logits, dim=1)
        return logits, probas

    
torch.manual_seed(random_seed)
model = MultilayerPerceptron(num_features=num_features,
                             num_classes=num_classes)

model = model.to(device)

optimizer = torch.optim.SGD(model.parameters(), lr=learning_rate)

In [104]:
def compute_accuracy(net, data_loader):
    net.eval()
    correct_pred, num_examples = 0, 0
    with torch.no_grad():
        for features, targets in data_loader:
            features = features.view(-1, 20*30).float().to(device)
            targets = targets.long().to(device)
            logits, probas = net(features)
            _, predicted_labels = torch.max(probas, 1)
            num_examples += targets.size(0)
            correct_pred += (predicted_labels == targets).sum()
        return correct_pred.float()/num_examples * 100
    

start_time = time.time()
for epoch in range(num_epochs):
    model.train()
    for batch_idx, (features, targets) in enumerate(train_loader):
        
        features = features.view(-1, 20*30).to(device)
        targets = targets.to(device)
            
        ### FORWARD AND BACK PROP
        logits, probas = model(features)
        cost = F.cross_entropy(logits, targets.long())
        optimizer.zero_grad()
        
        cost.backward()
        
        ### UPDATE MODEL PARAMETERS
        optimizer.step()
        
        ### LOGGING
        if not batch_idx % 50:
            print ('Epoch: %03d/%03d | Batch %03d/%03d | Cost: %.4f' %(epoch+1, num_epochs, batch_idx, len(train_loader), cost))

    with torch.set_grad_enabled(False):
        print('Epoch: %03d/%03d training accuracy: %.2f%%' % (epoch+1, num_epochs, compute_accuracy(model, train_loader)))
        print('Epoch: %03d/%03d validation accuracy: %.2f%%' % (epoch+1, num_epochs, compute_accuracy(model, val_loader)))
        
    print('Time elapsed: %.2f min' % ((time.time() - start_time)/60))
    
print('Total Training Time: %.2f min' % ((time.time() - start_time)/60))


Epoch: 001/012 | Batch 000/124 | Cost: 2.6312
Epoch: 001/012 | Batch 050/124 | Cost: 2.3747
Epoch: 001/012 | Batch 100/124 | Cost: 2.1833
Epoch: 001/012 training accuracy: 25.11%
Epoch: 001/012 validation accuracy: 14.40%
Time elapsed: 0.10 min
Epoch: 002/012 | Batch 000/124 | Cost: 2.1160
Epoch: 002/012 | Batch 050/124 | Cost: 2.1970
Epoch: 002/012 | Batch 100/124 | Cost: 2.0899
Epoch: 002/012 training accuracy: 33.44%
Epoch: 002/012 validation accuracy: 16.02%
Time elapsed: 0.21 min
Epoch: 003/012 | Batch 000/124 | Cost: 1.8181
Epoch: 003/012 | Batch 050/124 | Cost: 1.9824
Epoch: 003/012 | Batch 100/124 | Cost: 1.9607
Epoch: 003/012 training accuracy: 37.22%
Epoch: 003/012 validation accuracy: 16.84%
Time elapsed: 0.31 min
Epoch: 004/012 | Batch 000/124 | Cost: 1.5802
Epoch: 004/012 | Batch 050/124 | Cost: 1.8433
Epoch: 004/012 | Batch 100/124 | Cost: 2.0637
Epoch: 004/012 training accuracy: 42.66%
Epoch: 004/012 validation accuracy: 18.66%
Time elapsed: 0.41 min
Epoch: 005/012 | Bat

In [105]:
print('Test accuracy: %.2f%%' % (compute_accuracy(model, test_loader)))

Test accuracy: 21.75%


In [90]:
print('Test accuracy: %.2f%%' % (compute_accuracy(model, val_loader)))

Test accuracy: 22.52%
